In [1]:
import scanpy as sc
import scvelo as sv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sc.settings.set_figure_params(frameon=False, dpi=100)

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [3]:
summary_tab = pd.DataFrame(columns = ('Method', 'Dataset', 'Celltype', 'Mean Probability'))

In [4]:
datasets = ['Pancreas_with_cc', 'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow']

In [5]:
ground_truth = {
'HumanBoneMarrow' : [("HSC_1", "Ery_1"), ("HSC_1", "HSC_2"), ("Ery_1", "Ery_2")],
'DentateGyrus'  : [('nIPC', 'Neuroblast'), ('Neuroblast', 'Granule immature'), ('Granule immature', 'Granule mature'),
                 ("Radial Glia-like", "Astrocytes"), ("OPC", "OL")],
'Pancreas_with_cc' : [("Ngn3 high EP", "Pre-endocrine"), ("Pre-endocrine", "Alpha"), ("Pre-endocrine", "Beta"), ("Pre-endocrine", "Delta"), ("Pre-endocrine", "Epsilon")],
'MouseBoneMarrow' : [('dividing', 'progenitors'), ('progenitors', 'activating')],
'MouseErythroid' : [("Blood progenitors 1", "Blood progenitors 2"), ("Blood progenitors 2", "Erythroid1"),
                  ("Erythroid1", "Erythroid2"), ("Erythroid2", "Erythroid3")]
}

In [6]:
tab = pd.DataFrame(columns = ('Dataset', 'Transition', 'Time Difference'))
for j in range(len(datasets)):
    print(j)
    adata = sc.read_h5ad('/nfs/team283/aa16/data/fate_benchmarking/CellRank/' + 'cell2fate' + datasets[j] + 'AnnDataForCellRank.h5ad')
    for i in range(len(ground_truth[datasets[j]])):
        tab_temp = pd.DataFrame(columns = ('Dataset', 'Transition', 'Time Difference'), index = [str(i) + str(j)])
        tab_temp.loc[str(i) + str(j), 'Dataset'] = datasets[j]
        tab_temp.loc[str(i) + str(j), 'Transition'] = str(ground_truth[datasets[j]][i])
        tab_temp.loc[ str(i) + str(j), 'Time Difference'] = np.median(adata.obs['Time (hours)'][adata.obs['clusters'] == ground_truth[datasets[j]][i][1]]) - np.median(adata.obs['Time (hours)'][adata.obs['clusters'] == ground_truth[datasets[j]][i][0]])
        tab = pd.concat([tab, tab_temp], axis = 0)

0
1
2
3
4


In [8]:
tab.to_csv('/nfs/team283/aa16/data/fate_benchmarking/TimeDifferences.csv')

In [3]:
# Define custom effects
class ReadEffect:
    def __init__(self):
        pass

class WriteEffect:
    def __init__(self, value):
        self.value = value

# Define effect handlers
def handle_read(effect, continuation):
    return continuation.send("Hello")

def handle_write(effect, continuation):
    print(effect.value)
    return continuation

# Use effectful operations
def greet():
    x = yield ReadEffect()
    yield WriteEffect("Received: " + x)

# Run the program with effect handlers
def run(program):
    generator = program()
    try:
        effect = next(generator)
        while True:
            if isinstance(effect, ReadEffect):
                effect = generator.send("Hello")
            elif isinstance(effect, WriteEffect):
                effect = generator.send("World")
            else:
                raise TypeError(f"Unsupported effect: {effect}")
    except StopIteration:
        pass

# Run the program
run(greet)


In [4]:
run(greet)